In [1]:
from bia_bob._rag import HintVectorStore
from bia_bob._machinery import Context
from bia_bob import bob

In [2]:
bob("""
Load blobs.tif using aicsimageio
segment the bright blobs
show the result
""")

Hint lines in System prompt: 191


I will load the `blobs.tif` image using `aicsimageio`, segment the bright blobs, and display the result.



In [3]:
Context.hint_store = HintVectorStore()

In [4]:
bob("""
Load blobs.tif using aicsimageio
segment the bright blobs
show the result
""")

Hint lines in System prompt: 26


I will load the `blobs.tif` image using `aicsimageio`, segment the bright blobs, and display the result.

